# Miami Open Street Maps cleaning

Map Area: Miami, Florida, United States Map zen 
Map zen: [link](https://mapzen.com/data/metro-extracts/metro/miami_florida/)

OBS.: Para saber como replicar a limpeza, olhe [aqui](p3-cleaning-osm-data.html)


## 1. Problemas encontrados no seu mapa

Após baixar dos dados e executar uma análise inicial em uma amostra deles, encontrei alguns problemas como:

* Falta de padronização em mais de um atributo (tag)
* Falta de atributos
* Nós com descrição de estados diferentes do esperado (como Calífórnia e Wyoming)

### Falta de padronização

Um problema que ficou evidente desde o inicio da análise foi a falta de padronizão dos valores de várias tags como: 'addr:postcode', 'addr:state', 'addr:country' tanto para nós como para caminhos. Para o caso do estado e do país, eu defini valore padronizados e atualizei todos os valores errados para o padrão.

Para o código postal, a tarefa foi mais complexa. Essa tag possuia valores como:

* 33328 (valor esperado)
* 33483-4534
* FL 33166
* FL33401

Após uma pesquisa, consegui identificar que o formato que usa "5 digitos - 4 digitos" é uma versão extendida do valor esperado e consequentemente não é um formato errado.

Para corrigir o problema do código postal foi necessário apenas a utlização de uma expressão regular que encontra dentro do texto os formatos esperados.

### Falta de atributos

Com base na documentação ([link](http://wiki.openstreetmap.org/wiki/OSM_XML)), tanto um nó com um caminho deve possuir um atributo chamado *visible*. Entretanto, esse atributo não foi encontrado nos dados analisados. Isso deve indicar que esse atributo não é realmente obrigatório. 

Esse problema não é referente aos dados de forma direta, e sim a documentação que deveria deixar mais claro quais os atributos são realmente obrigatórios.


### Descrições incorretas

O problema que chamou mais atenção foi a presença de nós de outros estados nos dados. O estados esperado era Flórida, porém, encontrei nós dos estados da Califórnia e de Wyoming. Após uma análise mais específica desses nós, identifiquei outro fator estranho: as coordenadas geográficas desses nós idicavam que eles pertencem ao estado da Flórida.

Como não consegui identificar se o erro ocorreu no posicionamento geográfico ou na descrição do nó, decidi exclui-los dos dados.

### Problema de memória

Ao executar essa mesma análise inicial no conjunto completo dos dados, identifiquei outro problema, o consumo de memória. Ao utilizar o elemnttree eu perco o contexto do xml completo, logo se preciso realizar uma contagem de tags é necessário que eu guarde essa contagem em alguma variável. Por esse motivo e pelo fato da minha máquina não ser tão robusta, tive problema de desempenho chegando ao ponto de não conserguir terminar a execução dos scripts.

Para solucionar o problema citado, decidi dividir o script em iterações onde cada iteração salva arquivos de texto com informações que achei necessárias para a execução da prócima iteração.

## 2. Visão geral dos Dados

### Tamanho dos arquivos

![image](osm-size.PNG)

In [1]:
import pymongo

from pymongo import MongoClient
client = MongoClient("mongodb://localhost:27017")
db = client.osm

### Quantidade de documentos

In [2]:
db.miami.find().count()

2945303

### Quantidade de nós (*nodes*)

In [3]:
db.miami.find({"xml_tag_type":"node"}).count()

2615900

### Quantidade de caminhos (*ways*)

In [4]:
db.miami.find({"xml_tag_type":"way"}).count()

329403

### Quantidade de usuários únicos

In [5]:
len(db.miami.distinct("uid"))

1784

### Usuário que mais contribuiu

In [6]:
count_by_user = {"$group":{"_id":"$uid", "user" : { "$first": '$user' }, "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
top1 = {"$limit":1}

pipeline = [count_by_user, sort, top1]

list(db.miami.aggregate(pipeline))

[{u'_id': u'4346588', u'count': 854117, u'user': u'MiamiBuildingsImport'}]

### Quantidade de usuários que contribuiram apenas uma vez

In [7]:
count_by_user = {"$group":{"_id":"$uid", "count":{"$sum":1}}}
count_by_count = {"$group":{"_id":"$count", "num_users":{"$sum":1}}}
sort = {"$sort":{"_id":1}}
top1 = {"$limit":1}

pipeline = [count_by_user, count_by_count, sort, top1]

list(db.miami.aggregate(pipeline))

[{u'_id': 1, u'num_users': 536}]

### Onde relaxar nas férias?

Decidi passar as férias em Miami e como não gosto de caminhar quero saber qual o local (cidade que pertence a zona metropolitana de Miami) com maior variedade de serviços para que assim eu não precise me locomover muito dentro da cidade.

As 5 cidades com mais pontos marcados são:

In [26]:
match_node_tag = {"$match":{"xml_tag_type":"node"}}
has_city = {"$match":{'addr:city':{'$exists':'true'}}}
group_by_city = {"$group":{"_id":"$addr:city.v", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
top5 = {"$limit":5}

pipeline = [match_node_tag, has_city, group_by_city, sort, top5]

list(db.miami.aggregate(pipeline))

[{u'_id': u'Weston', u'count': 18201},
 {u'_id': u'Miami', u'count': 410},
 {u'_id': u'Miami Beach', u'count': 210},
 {u'_id': u'Wellington', u'count': 97},
 {u'_id': u'Coral Springs', u'count': 92}]

### Onde se divertir em Miami?

Decidi ir passar as férias em Miami novamente mas agora estou indo com a família. Por esse motivo estou procurando a cidade da região metropolitana de Miami com mais locais destinados ao lazer (no osm: "amenity").


As 5 cidades com mais marcações de *amenity* são:

In [28]:
has_city = {"$match":{'addr:city':{'$exists':'true'}}}
is_amenity = {"$match":{'amenity':{'$exists':'true'}}}
group_by_city = {"$group":{"_id":"$addr:city.v", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
top5 = {"$limit":5}

pipeline = [has_city, is_amenity, group_by_city, sort, top5]

list(db.miami.aggregate(pipeline))

[{u'_id': u'Miami', u'count': 147},
 {u'_id': u'Miami Beach', u'count': 104},
 {u'_id': u'Fort Lauderdale', u'count': 81},
 {u'_id': u'West Palm Beach', u'count': 47},
 {u'_id': u'Doral', u'count': 28}]

Sabendo que a cidade de Miami é minha melhor escolhar decidi verificar que tipos de lazer esses pontos oferecem.

In [33]:
is_miami = {"$match":{'addr:city.v':'Miami'}}
has_amenity = {"$match":{'amenity':{'$exists':'true'}}}
group_by_city = {"$group":{"_id":"$amenity.v", "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
top10 = {"$limit":10}

pipeline = [is_miami, is_amenity, group_by_city, sort, top10]

list(db.miami.aggregate(pipeline))

[{u'_id': u'restaurant', u'count': 37},
 {u'_id': u'fast_food', u'count': 21},
 {u'_id': u'cafe', u'count': 13},
 {u'_id': u'school', u'count': 10},
 {u'_id': u'pharmacy', u'count': 8},
 {u'_id': u'fuel', u'count': 5},
 {u'_id': u'hospital', u'count': 5},
 {u'_id': u'theatre', u'count': 5},
 {u'_id': u'bank', u'count': 5},
 {u'_id': u'library', u'count': 5}]

## 3. Outras ideias em relação aos conjuntos de dados

### Tipo do caminho

Com base na [wiki do osm](http://wiki.openstreetmap.org/wiki/Way), um caminho pode ser classificado em três tipos: *oneway* e *area*. Porém, a quantidade de caminhos com tal classificação é baixa:

In [ ]:
print "Oneway: ", db.miami.find({'oneway':{'$exists': 'true'}}).count()
print "Area: ", db.miami.find({'area':{'$exists': 'true'}}).count()

Uma forma de resolver esse problema é automatizar o processo de classificação dos caminhos. Calcular a próximidade do(s) último(s) nó(s) de um caminho com o(s) primeiro(s) é uma solução possível para tal problema.

### Contribuição

Como foi visto na seção 2 a quantidade de usuários que contribuíram para a base é bem pequena. Além disso é possível identificar que poucos usuários contribuem bem mais que o restante.

In [34]:
count_by_user = {"$group":{"_id":"$uid", "user" : { "$first": '$user' }, "count":{"$sum":1}}}
sort = {"$sort":{"count":-1}}
top1 = {"$limit":10}

pipeline = [count_by_user, sort, top1]

list(db.miami.aggregate(pipeline))

[{u'_id': u'4346588', u'count': 854117, u'user': u'MiamiBuildingsImport'},
 {u'_id': u'369983', u'count': 352895, u'user': u'grouper'},
 {u'_id': u'2533093', u'count': 263532, u'user': u'carciofo'},
 {u'_id': u'147510', u'count': 226299, u'user': u'woodpeck_fixbot'},
 {u'_id': u'2318', u'count': 135934, u'user': u'Latze'},
 {u'_id': u'429761', u'count': 116219, u'user': u'freebeer'},
 {u'_id': u'451693', u'count': 54646, u'user': u'bot-mode'},
 {u'_id': u'207745', u'count': 50627, u'user': u'NE2'},
 {u'_id': u'550560', u'count': 50383, u'user': u'Seandebasti'},
 {u'_id': u'230350', u'count': 45007, u'user': u'westendguy'}]

Acredito que existam duas vertentes que podem ser focadas:
1. Fazer com que os usuários contribuam mais
2. Fazer que novas pessoas se tornem usuários

Uma solução para o ponto 1 é a introdução de recompensas para cada contribuição realizada de forma que quanto mais o usuário ajude o sistema mais ele vai conseguir usufruir do serviço.

O sistema de recompensas também pode ajudar o ponto 2. Entretanto, acredito isso deve ser feito em conjunto com outras ferramentas mais difundidas. Por exemplo, se o Facebook passa usar o OSM como serviço de mapas e o usuário que marcar uma localização que não existe ele seria motivado a adicionar tal localização no OSM.

#### Problema do aumento de usuários

De certa maneira o aumento da quantidade de usuário contribuindo é um aspecto positivo, porém, isso pode trazer uma problema perigoso: informação errada. Quanto maior a quantidade de pessoas colaborando, maior a chance de existir problemas causados pelo erro humano (algumas vezes intencionais).  Esse problema pode ser atacado com a implantação de um sistema de validação: sempre que um usuário considerar que uma informação está errada ele pode marcar essa informação como incorreta e sugerir como corrigi-la. Assim que alguma informação atingir um limiar de marcações ela seria atualizada.

Outro problema que pode ser criado pelo aumento de usuários contribuindo é a falta de padronização da informação. Para solucionar esse problema poderiam ser criados *templates* para informações mais recorrentes. Por exemplo: se o usuário vai adicionar um restaurante ao OSM ele poderia receber a sugestão de usar o *template restaurant*, o *template amenity* ou em último caso o usuário pode decidir não usar nenhum template. A criação dos templates também pode ser colaborativa e só os templates mais votados seriam sugeridos para o usuário. 